In [1]:
import os
import numpy as np
import pandas as pd

from sys import path
path.append("../analysis/utils/")

from utils import get_datasets

In [2]:
DATASETS = ["webkb", "20ng", "reut", "acm"]

CLFS = ["kpr", "ktr", "lpr", "ltr", "sfr", "stmk", "xfr", "xpr", "xtr", "kfr", "ktmk", "lfr", "ltmk", "spr", "str", "xlnet_softmax", "xtmk", "rep_bert"]

DATA_SOURCE = "/home/welton/data"

ORACLE_DIR = f"{DATA_SOURCE}/oracle/upper_bound"

In [3]:
pd_datasets = get_datasets(DATASETS, path=f"{DATA_SOURCE}/pd_datasets/__dset__.csv", sep=';')

In [14]:
# For each dataset.
for dset in DATASETS:
    df = pd_datasets[dset]
    labels = df[["label", "fold_id"]]
    try:
        preds = df.drop(columns=["label", "fold_id", "rep_bert_bkp", "bert"])
    except:
        preds = df.drop(columns=["label", "fold_id", "bert"])
    # For each classifier.
    for clf in CLFS:
        # For each fold.
        for fold in np.arange(10):

            train_load = np.load(f"/home/welton/data/clfs_output/split_10/{dset}/10_folds/{clf}/{fold}/train.npz")
            train_preds = train_load["X_train"].argmax(axis=1)
            train_labels = np.load(f"/home/welton/data/datasets/labels/split_10/{dset}/{fold}/train.npy")
            
            test_load = np.load(f"/home/welton/data/clfs_output/split_10/{dset}/10_folds/{clf}/{fold}/test.npz")
            test_preds = test_load["X_test"].argmax(axis=1)
            test_labels = np.load(f"/home/welton/data/datasets/labels/split_10/{dset}/{fold}/test.npy")

            # Making train upper bound.
            train_upper = np.zeros(train_preds.shape[0])
            
            # Set Hit/True (value 1) on documents right classifieds.
            train_upper[train_preds == train_labels] = 1
            output_dir = f"{ORACLE_DIR}/{dset}/{clf}/{fold}/"
            os.makedirs(output_dir, exist_ok=True)
            output = f"{output_dir}/train"
            # Save the upper_ground.
            np.savez(output, y=train_upper)

            # Making test upper bound.
            test_upper = np.zeros(test_preds.shape[0])

            # Set Hit/True (value 1) on documents right classifieds.
            test_upper[test_preds == test_labels] = 1
            output = f"{output_dir}/test"
            # Save the upper_ground.
            np.savez(output, y=test_upper)

            # Generating upper test and train test.
            output_dir = f"{ORACLE_DIR.replace('bound', 'train')}/{dset}/{clf}/{fold}/"
            os.makedirs(output_dir, exist_ok=True)
            output = f"{output_dir}/train"
            np.savez(output, y=train_upper)
            output = f"{output_dir}/test"
            fake_test = np.zeros(test_upper.shape[0]) + 1
            np.savez(output, y=fake_test)

            output_dir = f"{ORACLE_DIR.replace('bound', 'test')}/{dset}/{clf}/{fold}/"
            os.makedirs(output_dir, exist_ok=True)
            output = f"{output_dir}/train"
            fake_train = np.zeros(train_upper.shape[0]) + 1
            np.savez(output, y=fake_train)
            output = f"{output_dir}/test"
            np.savez(output, y=test_upper)
            

In [28]:
ct = np.load("/home/welton/data/oracle/upper_test/webkb/rep_bert/0/train.npz")["y"]
ct2 = np.load("/home/welton/data/oracle/upper_test/webkb/ltr/0/train.npz")["y"]
np.unique(ct2 == ct)

array([ True])

In [29]:
y = np.load("/home/welton/data/datasets/labels/split_10/webkb/1/train.npy")
yc = np.load("/home/welton/data/clfs_output/split_10/webkb/10_folds/ltr/1/train.npz")['y_train']
np.unique(y == yc)

array([ True])

In [27]:
np.multiply([[1,0,1], [0,0,0], [1,1,1]], [[2,2,2],[2,2,2],[2,2,2]])

array([[2, 0, 2],
       [0, 0, 0],
       [2, 2, 2]])